In [38]:
import os
import numpy as np
import pandas as pd

In [39]:
DATASET_FOLDER = './dataset/'
# test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
# sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
# sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

In [40]:
test.head()

,sample_id,catalog_content,image_link
0,100179,Item Name: Rani 14-Spice Eshamaya's Mango Chut...,https://m.media-amazon.com/images/I/71hoAn78AW...
1,245611,Item Name: Natural MILK TEA Flavoring extract ...,https://m.media-amazon.com/images/I/61ex8NHCIj...
2,146263,Item Name: Honey Filled Hard Candy - Bulk Pack...,https://m.media-amazon.com/images/I/61KCM61J8e...
3,95658,Item Name: Vlasic Snack'mm's Kosher Dill 16 Oz...,https://m.media-amazon.com/images/I/51Ex6uOH7y...
4,36806,"Item Name: McCormick Culinary Vanilla Extract,...",https://m.media-amazon.com/images/I/71QYlrOMoS...


In [41]:
n = test.shape[0]  # 75000
example_indices = list(map(int, np.random.choice(n, size=5, replace=False)))
example_indices

[60767, 1295, 63632, 37361, 9983]

In [42]:
for i in example_indices:
    print(f"Example {i} of {n}")
    print(test.at[i, "catalog_content"])
    # print(test.iloc[i])
    print()

Example 60767 of 75000
Item Name: Satya+Sage Luxury Aesthetic Scented Candle (Meditation, 9oz – Amber, Incense, 50 Hours) | All Natural, Non-Toxic Coconut and Vegan Soy Wax w/Essential Oils for Massage, Relaxation | Hand Poured
Bullet Point 1: ✔️ MEDITATION SCENT, AMBER, INCENSE, 9OZ – Meditation from Satya + Sage has rich resins and sultry ambers that partner with the smoky incense notes. . This 9oz candle jar will burn for UP TO 50 HOURS of olfactory enjoyment.
Bullet Point 2: ✔️ SAFE FOR YOU AND THE ENVIRONMENT – At Satya + Sage, we use only all-natural and non-toxic ingredients so that what you burn in your space won’t have negative health impacts. Our candles are made from vegan coconut and soy wax, with a cotton wick, and are always dye, zinc, and phthalate free. Our packaging is made from FSC certified material for minimal impact on our forests, and our jars are 100% recyclable and refillable glass.
Bullet Point 3: ✔️ PREMIUM-GRADE ESSENTIAL OIL FRAGRANCES – We do not believe in

In [43]:
import re 


# regex to extract "Item Name: ...\n" (case-insensitive, allow end-of-string)
_brand_name_pattern = r'(?i)Item Name:\s*(.*?)\s*(?:\n|$)'
_value_pattern = r'(?i)Value:\s*(.*?)\s*(?:\n|$)'
_unit_pattern = r'(?i)Unit:\s*(.*?)\s*(?:\n|$)'

def _extract_item_name(text):
    if pd.isna(text):
        return np.nan
    m = re.search(_brand_name_pattern, str(text))
    return m.group(1).strip() if m else np.nan

def _extract_value(text):
    if pd.isna(text):
        return np.nan
    m = re.search(_value_pattern, str(text))
    return m.group(1).strip() if m else np.nan

def _extract_unit(text):
    if pd.isna(text):
        return np.nan
    m = re.search(_unit_pattern, str(text))
    return m.group(1).strip() if m else np.nan

test['item_name'] = test['catalog_content'].apply(_extract_item_name)
test['value'] = test['catalog_content'].apply(_extract_value)
test['unit'] = test['catalog_content'].apply(_extract_unit)

In [44]:
test.head()

,sample_id,catalog_content,image_link,item_name,value,unit
0,100179,Item Name: Rani 14-Spice Eshamaya's Mango Chut...,https://m.media-amazon.com/images/I/71hoAn78AW...,Rani 14-Spice Eshamaya's Mango Chutney (Indian...,10.5,Ounce
1,245611,Item Name: Natural MILK TEA Flavoring extract ...,https://m.media-amazon.com/images/I/61ex8NHCIj...,Natural MILK TEA Flavoring extract by HALO PAN...,2.0,Fl Oz
2,146263,Item Name: Honey Filled Hard Candy - Bulk Pack...,https://m.media-amazon.com/images/I/61KCM61J8e...,Honey Filled Hard Candy - Bulk Pack 2 Pounds -...,32.0,Ounce
3,95658,Item Name: Vlasic Snack'mm's Kosher Dill 16 Oz...,https://m.media-amazon.com/images/I/51Ex6uOH7y...,Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2),2.0,Count
4,36806,"Item Name: McCormick Culinary Vanilla Extract,...",https://m.media-amazon.com/images/I/71QYlrOMoS...,"McCormick Culinary Vanilla Extract, 32 fl oz -...",32.0,Fl Oz


In [45]:
test['unit'].value_counts()

unit
Ounce     40921
Count     17458
Fl Oz     11171
ounce      1963
None       1011
          ...  
Tin           1
Liter         1
lbs           1
Pieces        1
Lbs           1
Name: count, Length: 110, dtype: int64

RuntimeError: Could not find 'catalog_cotent' as a DataFrame column, Series, or list in the notebook namespace.

In [ ]:
def normalize_unit_to_ounce(x):
    # preserve NaNs
    if pd.isna(x):
        return x
    s = str(x).strip()
    # match "oz", "oz.", "ounce", "ounces" (case-insensitive)
    if re.search(r'\boz\.?\b|\bounces?\b', s, flags=re.I):
        return 'ounce'
    return s

# apply to test dataframe
test['unit'] = test['unit'].apply(normalize_unit_to_ounce)

In [ ]:
test.head()

,sample_id,catalog_content,image_link,item_name,value,unit
0,100179,Item Name: Rani 14-Spice Eshamaya's Mango Chut...,https://m.media-amazon.com/images/I/71hoAn78AW...,Rani 14-Spice Eshamaya's Mango Chutney (Indian...,10.5,ounce
1,245611,Item Name: Natural MILK TEA Flavoring extract ...,https://m.media-amazon.com/images/I/61ex8NHCIj...,Natural MILK TEA Flavoring extract by HALO PAN...,2.0,ounce
2,146263,Item Name: Honey Filled Hard Candy - Bulk Pack...,https://m.media-amazon.com/images/I/61KCM61J8e...,Honey Filled Hard Candy - Bulk Pack 2 Pounds -...,32.0,ounce
3,95658,Item Name: Vlasic Snack'mm's Kosher Dill 16 Oz...,https://m.media-amazon.com/images/I/51Ex6uOH7y...,Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2),2.0,Count
4,36806,"Item Name: McCormick Culinary Vanilla Extract,...",https://m.media-amazon.com/images/I/71QYlrOMoS...,"McCormick Culinary Vanilla Extract, 32 fl oz -...",32.0,ounce


In [ ]:
test['unit'].unique()

array(['ounce', 'Count', 'None', 'count', 'each', 'Box', 'pound', 'packs',
       'Pack', 'milliliter',
       '200 Capsules - Packed Bulk 20 packs of 10 capsules per vacuum sealed pouch',
       'gram', 'Packet', 'Pound', 'product_weight', 'grams', 'lb', '1',
       'KIT', 'NA', 'fluid_ounces', 'kg', 'tea bags', 'Tea Bags',
       'Liters', 'Packs', 'ct', 'millilitro', '2', 'BAG', 'Gram',
       'Each / Pack: 1', 'Container', 'pac', 'Milliliters', 'Foot', 'CT',
       'Paper Cupcake Liners', 'Each', 'COUNT', 'Bag', 'LB', 'millilitre',
       'K-Cups', 'M', 'Sq Ft', 'Bottle', 'bottle', 'g', '0.7 LBS</p>',
       'PACK', 'Ltr', 'EA', 'ml', '....', 'unit', 'SACHET', 'ea',
       '(Pack of 1)', 'Per Package', 'Jar', 'Tin', 'Liter', 'lbs',
       'Pieces', 'Piece', 'Case / Pack: 6', 'bag', 'Gallon', 'JARS',
       'BOX', 'Count / Count', 'gallons', 'Pounds', 'Cou',
       'Sugar Substitute', 'Pouch', 'Capsules', 'stück', 'jar',
       'Comes as a single dispenser unit with 1 count of candy

In [ ]:
import pandas as pd
import re

# ✅ Define normalization map (from our cleaned grouping)
unit_map = {
    # Weight
    'gram': 'Gram', 'grams': 'Gram', 'gramm': 'Gram', 'gr': 'Gram', 'Gram': 'Gram', 'Grams(gm)': 'Gram',
    'kg': 'Kilogram',
    'pound': 'Pound', 'pounds': 'Pound', 'Pound': 'Pound', 'LB': 'Pound', 'lb': 'Pound',

    # Volume
    'ml': 'Milliliter', 'millilitre': 'Milliliter', 'milliliter': 'Milliliter',
    'mililitro': 'Milliliter', 'Liters': 'Liter', 'ltr': 'Liter', '2.5 gal.': 'Gallon',

    # Count / Quantity
    'Count': 'Count', 'count': 'Count', 'COUNT': 'Count', 'ct': 'Count', 'CT': 'Count',
    'units': 'Count', 'unità': 'Count',
    'Each': 'Each', 'each': 'Each',
    'Pack': 'Pack', 'pack': 'Pack', 'Packs': 'Pack', 'packs': 'Pack', 'PACK': 'Pack',
    'Per Package': 'Pack', 'per Box': 'Box', 'per Carton': 'Box',
    'Box': 'Box', 'box': 'Box', 'BOX/12': 'Box', 'Carton': 'Box', 'CASE': 'Box',
    'Bag': 'Bag', 'bag': 'Bag', 'Ziplock bags': 'Bag',
    'Bottle': 'Bottle', 'bottle': 'Bottle', 'bottles': 'Bottle',
    'Can': 'Can', 'can': 'Can', 'Pouch': 'Pouch', 'Piece': 'Piece',
    'K-Cups': 'K-Cups', 'Jar': 'Jar', 'capsule': 'capsule',
    'Tea Bags': 'Tea Bags', 'Tea bags': 'Tea Bags', 'Paper Cupcake Liners': 'Paper Cupcake Liners',
    'Bucket': 'Bucket',

    # Area / Length
    'sq ft': 'Square Foot', 'Sq Ft': 'Square Foot', 'Foot': 'Foot', 'in': 'Inch'
}


def normalize_unit(x):
    """Normalize the 'unit' column based on standard mapping and ounce matching."""
    if pd.isna(x):
        return x

    s = str(x).strip()

    # ✅ Match ounce variations (extra logic)
    if re.search(r'\boz\.?\b|\bounces?\b', s, flags=re.I):
        return 'Ounce'

    # ✅ Normalize case-insensitively using map
    s_lower = s.lower()
    for k, v in unit_map.items():
        if s_lower == k.lower():
            return v

    # ✅ Default: return original cleaned string
    return s


# 🧹 Apply normalization
test['unit'] = test['unit'].apply(normalize_unit)

In [ ]:
test['unit'].unique()

array(['Ounce', 'Count', 'None', 'Each', 'Box', 'Pound', 'Pack',
       'Milliliter',
       '200 Capsules - Packed Bulk 20 packs of 10 capsules per vacuum sealed pouch',
       'Gram', 'Packet', 'product_weight', '1', 'KIT', 'NA',
       'fluid_ounces', 'Kilogram', 'Tea Bags', 'Liter', 'millilitro', '2',
       'Bag', 'Each / Pack: 1', 'Container', 'pac', 'Milliliters', 'Foot',
       'Paper Cupcake Liners', 'K-Cups', 'M', 'Square Foot', 'Bottle',
       'g', '0.7 LBS</p>', 'EA', '....', 'unit', 'SACHET', 'ea',
       '(Pack of 1)', 'Jar', 'Tin', 'lbs', 'Pieces', 'Piece',
       'Case / Pack: 6', 'Gallon', 'JARS', 'Count / Count', 'gallons',
       'Cou', 'Sugar Substitute', 'Pouch', 'Capsules', 'stück',
       'Comes as a single dispenser unit with 1 count of candy refills',
       '12.54', 'Lbs'], dtype=object)

In [ ]:
# Define invalid or garbage entries
invalid_units = [
    'None', '', '-', '---', '1', '8', '24',
    'product_weight',
    'Each / Pack: 1',
    '1 PK. Color(s): -Black. Product Type: -Permanent. Pack Quantity: -1. Tip Type: -Chisel. Dimensions: Overall Product Weight: -0.06 lbs.',
    'cm/inch) ??Size:S---US:4---Bust:96cm/37.80---Length:66cm/25.98---Shoulder to sleeve:63cm/24.80 ??Size:M---US:6---Bust:100cm/39.37---Length:67cm/26.38---Shoulder to sleeve:64cm/25.20 ??Size:L---US:8---Bust:104cm/40.94---Length:68cm/26.77---Shoulder to sleeve:65cm/25.59 ??Size:XL---US:10---Bust:108cm/42.52---Length:69cm/27.17---Shoulder to sleeve:66cm/25.98 ??Size:XXL---US:12---Bust:112cm/44.09---Length:70cm/27.56---Shoulder to sleeve:67cm/26.38 ??Size:XXXL---US:14---Bust:116cm/45.67---Length:70cm/27.56---Shoulder to sleeve:68cm/26.77'
]

def clean_invalid_units(x):
    """Return None if x is an invalid or meaningless unit."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s in invalid_units:
        return None
    return s

# 🧹 Apply the filter
test['unit'] = test['unit'].apply(clean_invalid_units)

In [ ]:
test['unit'].value_counts()

unit
Ounce                                                                         55327
Count                                                                         18226
Pound                                                                           219
Pack                                                                             38
Gram                                                                             35
Each                                                                             17
Milliliter                                                                       17
Liter                                                                             9
Bottle                                                                            9
Bag                                                                               7
Box                                                                               5
Jar                                                                    

In [ ]:
# define item-type units to merge into "Count"
count_like_units = [
    'Pack', 'Each', 'Box', 'Bottle', 'Can', 'Pouch', 'Bag', 'Jar',
    'K-Cups', 'Piece', 'Tea Bags', 'Bucket', 'capsule', 'Paper Cupcake Liners'
]

def merge_packaging_units(x):
    """Merge packaging / count-like units into 'Count'."""
    if pd.isna(x):
        return x
    s = str(x).strip()
    if s in count_like_units:
        return 'Count'
    return s

# 🧹 Apply transformation
test['unit'] = test['unit'].apply(merge_packaging_units)

In [ ]:
test['unit'].value_counts()

unit
Ounce                                                                         55327
Count                                                                         18319
Pound                                                                           219
Gram                                                                             35
Milliliter                                                                       17
Liter                                                                             9
Foot                                                                              4
Packet                                                                            3
Gallon                                                                            2
pac                                                                               2
Square Foot                                                                       2
EA                                                                     

In [ ]:
test.to_csv('./dataset/test_cleaned_units.csv', index=False)

In [ ]:
# Check the length of the dataset  
len(test)

75000

In [ ]:
test['unit'].isna().sum()
test['unit'].isnull().sum()

np.int64(1027)

In [ ]:
test.head()

,sample_id,catalog_content,image_link,item_name,value,unit
0,100179,Item Name: Rani 14-Spice Eshamaya's Mango Chut...,https://m.media-amazon.com/images/I/71hoAn78AW...,Rani 14-Spice Eshamaya's Mango Chutney (Indian...,10.5,Ounce
1,245611,Item Name: Natural MILK TEA Flavoring extract ...,https://m.media-amazon.com/images/I/61ex8NHCIj...,Natural MILK TEA Flavoring extract by HALO PAN...,2.0,Ounce
2,146263,Item Name: Honey Filled Hard Candy - Bulk Pack...,https://m.media-amazon.com/images/I/61KCM61J8e...,Honey Filled Hard Candy - Bulk Pack 2 Pounds -...,32.0,Ounce
3,95658,Item Name: Vlasic Snack'mm's Kosher Dill 16 Oz...,https://m.media-amazon.com/images/I/51Ex6uOH7y...,Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2),2.0,Count
4,36806,"Item Name: McCormick Culinary Vanilla Extract,...",https://m.media-amazon.com/images/I/71QYlrOMoS...,"McCormick Culinary Vanilla Extract, 32 fl oz -...",32.0,Ounce


In [ ]:
test['value'].isna().sum()
test['value'].isnull().sum()

np.int64(0)